# AWS AI Agent - Deployed Endpoint Demo

This notebook demonstrates the **complete user acceptance criteria** for the AWS AI Agent assessment.

## 📋 What This Notebook Demonstrates

1. ✅ **Cognito Authentication** - Creating users and obtaining JWT tokens
2. ✅ **AgentCore Invocation** - Using **HTTPS + Cognito JWT Bearer token**
3. ✅ **All 5 User Acceptance Queries** - Required assessment queries
4. ✅ **Langfuse Observability** - Automatic tracing of all executions
5. ✅ **LangGraph ReAct Agent** - Multi-step reasoning with tool use
6. ✅ **RAG with Bedrock KB** - Searching Amazon financial documents
7. ✅ **Stock Data Tools** - Real-time and historical queries via yfinance

## 🔧 Prerequisites

Before running this notebook:

1. **Deploy infrastructure**: Run `terraform apply` in the `terraform/` directory
2. **Upload documents**: Run `./scripts/upload_financial_documents.sh`
3. **Update configuration**: Set the correct values below from terraform outputs:
   - `COGNITO_USER_POOL_ID` - from `terraform output cognito_user_pool_id`
   - `COGNITO_APP_CLIENT_ID` - from `terraform output cognito_app_client_id`
   - `AGENTCORE_RUNTIME_ID` - from `terraform output agentcore_runtime_id`
4. **Configure AWS credentials**: Ensure AWS CLI is configured with valid credentials

## 📦 Required Python Packages

```bash
pip install boto3 requests
```

---

## Setup & Configuration

In [150]:
import boto3
import requests
import json
import uuid
import urllib.parse
from typing import Dict, Any
import time
from IPython.display import display, Markdown

# Deployment configuration - UPDATE THESE FROM TERRAFORM OUTPUTS
AWS_REGION = "us-east-2"
COGNITO_USER_POOL_ID = "us-east-2_FqySaFWQS"  # terraform output cognito_user_pool_id
COGNITO_APP_CLIENT_ID = "2dhmq9u19aho743foek2pvb15m"  # terraform output cognito_app_client_id
AGENTCORE_RUNTIME_ARN = "arn:aws:bedrock-agentcore:us-east-2:302703038631:runtime/dev_ai_agent_runtime-02UaUs5ZCH"  # terraform output agentcore_runtime_arn

# Construct AgentCore HTTPS endpoint
ESCAPED_ARN = urllib.parse.quote(AGENTCORE_RUNTIME_ARN, safe='')
AGENTCORE_ENDPOINT = f"https://bedrock-agentcore.{AWS_REGION}.amazonaws.com/runtimes/{ESCAPED_ARN}/invocations?qualifier=DEFAULT"

# Initialize boto3 client for Cognito
cognito_client = boto3.client('cognito-idp', region_name=AWS_REGION)

display(Markdown(
    f"**Configuration loaded**\n\n"
    f"| Setting | Value |\n"
    f"|---------|-------|\n"
    f"| Region | `{AWS_REGION}` |\n"
    f"| Cognito User Pool | `{COGNITO_USER_POOL_ID}` |\n"
    f"| Cognito App Client | `{COGNITO_APP_CLIENT_ID}` |\n"
    f"| AgentCore Runtime | `{AGENTCORE_RUNTIME_ARN.split('/')[-1]}` |\n"
))

**Configuration loaded**

| Setting | Value |
|---------|-------|
| Region | `us-east-2` |
| Cognito User Pool | `us-east-2_FqySaFWQS` |
| Cognito App Client | `2dhmq9u19aho743foek2pvb15m` |
| AgentCore Runtime | `dev_ai_agent_runtime-02UaUs5ZCH` |


## Cognito Authentication

### Step 1: Create Test User

In [151]:
def create_test_user(username: str = "test", password: str = "TestPassword123!", email: str = "test@example.com"):
    """Create a test user in Cognito user pool."""
    try:
        cognito_client.admin_create_user(
            UserPoolId=COGNITO_USER_POOL_ID,
            Username=username,
            UserAttributes=[
                {'Name': 'email', 'Value': email},
                {'Name': 'email_verified', 'Value': 'true'}
            ],
            MessageAction='SUPPRESS'
        )
        cognito_client.admin_set_user_password(
            UserPoolId=COGNITO_USER_POOL_ID,
            Username=username,
            Password=password,
            Permanent=True
        )
        display(Markdown(f"User **{username}** created successfully"))
        return True
    except cognito_client.exceptions.UsernameExistsException:
        display(Markdown(f"User **{username}** already exists (ready to use)"))
        return True
    except Exception as e:
        display(Markdown(f"**Error:** `{e}`"))
        return False

create_test_user()

User **test** already exists (ready to use)

True

### Step 2: Authenticate and Get JWT Token

**This JWT token will be used for AgentCore authorization via Bearer token.**

In [152]:
def authenticate(username: str = "test", password: str = "TestPassword123!") -> str:
    """Authenticate with Cognito and get JWT access token."""
    response = cognito_client.initiate_auth(
        ClientId=COGNITO_APP_CLIENT_ID,
        AuthFlow='USER_PASSWORD_AUTH',
        AuthParameters={'USERNAME': username, 'PASSWORD': password}
    )
    
    access_token = response['AuthenticationResult']['AccessToken']
    expires_in = response['AuthenticationResult']['ExpiresIn']
    
    display(Markdown(
        f"Authenticated as **{username}**\n\n"
        f"| Field | Value |\n"
        f"|-------|-------|\n"
        f"| Token type | `Bearer` |\n"
        f"| Expires in | {expires_in}s ({expires_in//60} min) |\n"
        f"| Token preview | `{access_token[:40]}...` |"
    ))
    return access_token

access_token = authenticate()

Authenticated as **test**

| Field | Value |
|-------|-------|
| Token type | `Bearer` |
| Expires in | 3600s (60 min) |
| Token preview | `eyJraWQiOiJ5V0p6YUV6UDVvQTU2d0k3Z2lOY0R4...` |

## AgentCore Invocation with Cognito JWT

This function invokes AgentCore via **HTTPS with Cognito JWT Bearer token**.

**Authentication Method:** OAuth (Cognito JWT) - matches AgentCore configuration

In [153]:
from IPython.display import display, Markdown, HTML

def query_agent(query: str, token: str, session_id: str = None, stream: bool = False) -> Dict[str, Any]:
    """Query the AI agent via AgentCore HTTPS endpoint with Cognito JWT."""
    if session_id is None:
        session_id = str(uuid.uuid4())

    mode = "Streaming" if stream else "Non-streaming"
    display(Markdown(f"---\n### Query\n> {query}\n\n`Session: {session_id}` | `Mode: {mode}`"))

    try:
        start = time.time()

        headers = {
            'Authorization': f'Bearer {token}',
            'Content-Type': 'application/json',
            'X-Session-Id': session_id
        }

        payload = {"query": query, "stream": stream}

        if stream:
            response = requests.post(
                AGENTCORE_ENDPOINT,
                headers=headers,
                json=payload,
                timeout=300,
                stream=True
            )

            if response.status_code != 200:
                error_msg = f"HTTP {response.status_code}: {response.text}"
                display(Markdown(f"**Error:** `{error_msg}`"))
                raise Exception(error_msg)

            elapsed_first = None
            full_answer = ""
            events = []
            tools_used = []

            print("Streaming response:\n")

            for line in response.iter_lines(decode_unicode=True):
                if elapsed_first is None:
                    elapsed_first = (time.time() - start) * 1000

                if line and line.startswith("data: "):
                    data_str = line[6:]
                    try:
                        event = json.loads(data_str)
                        events.append(event)
                        event_type = event.get("event_type", "unknown")
                        data = event.get("data", {})

                        if event_type == "tool_call":
                            tool_name = data.get("tool", "unknown")
                            tools_used.append(tool_name)
                            print(f"\n  >> Tool: {tool_name}({data.get('args', {})})", flush=True)
                        elif event_type == "tool_execution":
                            print(f"  >> Executing tool...", flush=True)
                        elif event_type == "agent_step":
                            pass  # intermediate step, don't print
                        elif event_type == "token":
                            token_text = data.get("content", "")
                            print(token_text, end="", flush=True)
                            full_answer += token_text
                        elif event_type == "final_answer":
                            answer_text = data.get("answer", "")
                            if answer_text and not full_answer:
                                # No tokens were streamed, print the full answer
                                print(answer_text, flush=True)
                            full_answer = answer_text or full_answer
                        elif event_type == "done":
                            pass
                        else:
                            print(f"\n  [{event_type}]: {data}", flush=True)
                    except json.JSONDecodeError:
                        pass

            elapsed = (time.time() - start) * 1000
            print(f"\n\n---")
            
            meta = f"Completed in {elapsed:.0f}ms"
            if elapsed_first is not None:
                meta += f" (first byte: {elapsed_first:.0f}ms)"
            meta += f" | {len(events)} events"
            if tools_used:
                meta += f" | Tools: {', '.join(tools_used)}"
            print(meta)

            return {
                'answer': full_answer,
                'events': events,
                'tools_used': tools_used,
                'session_id': session_id,
                'execution_time_ms': elapsed,
                'status_code': response.status_code
            }
        else:
            response = requests.post(
                AGENTCORE_ENDPOINT,
                headers=headers,
                json=payload,
                timeout=120
            )

            elapsed = (time.time() - start) * 1000

            if response.status_code != 200:
                error_msg = f"HTTP {response.status_code}: {response.text}"
                display(Markdown(f"**Error:** `{error_msg}`"))
                raise Exception(error_msg)

            result = response.json()

            answer = result.get('answer', '')
            sources = result.get('sources', [])
            trace_id = result.get('trace_id')
            trace_url = result.get('trace_url')
            exec_time = result.get('execution_time_ms', elapsed)
            reasoning = result.get('reasoning_steps', [])

            # Display reasoning steps
            if reasoning:
                steps_md = "\n".join([
                    f"- **Step {s.get('step_number', '?')}** — `{s.get('action', '')}`: "
                    f"{s.get('observation', '')[:150]}"
                    for s in reasoning
                ])
                display(Markdown(f"**Reasoning Steps:**\n{steps_md}"))

            # Display answer
            display(Markdown(f"### Answer\n\n{answer}"))

            # Display metadata
            meta_parts = [f"*{exec_time:.0f}ms*"]
            if sources:
                meta_parts.append(f"Sources: {', '.join(sources)}")
            if trace_url:
                meta_parts.append(f"[Langfuse Trace]({trace_url})")
            elif trace_id:
                meta_parts.append(f"Trace: `{trace_id}`")
            display(Markdown(" | ".join(meta_parts)))

            return {
                'answer': answer,
                'sources': sources,
                'trace_id': trace_id,
                'trace_url': trace_url,
                'reasoning_steps': reasoning,
                'session_id': session_id,
                'execution_time_ms': exec_time,
                'status_code': response.status_code
            }

    except requests.exceptions.RequestException as e:
        display(Markdown(f"**Request failed:** `{e}`"))
        raise

print("Helper function loaded")

Helper function loaded


---

# User Acceptance Queries

The following 5 queries validate all assessment requirements:

1. **Real-time stock price** → `retrieve_realtime_stock_price`
2. **Historical stock data** → `retrieve_historical_stock_price`
3. **Stock + analyst predictions** → Multi-tool with RAG
4. **AMZN price + AI business** → Complex multi-tool
5. **Office space data** → `search_financial_documents`

**All queries use Cognito JWT authentication via Bearer token** ✅

---

## Query 1: Current Stock Price

**Question**: "What is the stock price for Amazon right now?"

**Expected Tool**: `retrieve_realtime_stock_price`

In [156]:
result_1 = query_agent(
    "What is the stock price for Amazon right now?", 
    token=access_token,
    stream=True,
)

---
### Query
> What is the stock price for Amazon right now?

`Session: cbf51b86-1757-4e2a-aebe-9991f83a77f3` | `Mode: Streaming`

Streaming response:


  >> Tool: retrieve_realtime_stock_price({'symbol': 'AMZN'})
  >> Executing tool...
Based on the real-time data, Amazon (AMZN) is currently trading at $198.79 per share. Today's trading range has been between $197.28 (low) and $201.16 (high), with the stock opening at $198.99. The trading volume today is 66,321,593 shares, and Amazon's current market capitalization is approximately $2.13 trillion.


---
Completed in 7510ms (first byte: 4341ms) | 5 events | Tools: retrieve_realtime_stock_price


## Query 2: Historical Stock Prices (Q4 Last Year)

**Question**: "What were the stock prices for Amazon in Q4 last year?"

**Expected Tool**: `retrieve_historical_stock_price`

In [157]:
result_2 = query_agent(
    "What were the stock prices for Amazon in Q4 last year?",
    token=access_token,
    stream=True,
)

---
### Query
> What were the stock prices for Amazon in Q4 last year?

`Session: 8eaa56fb-5f58-463d-b871-b04f26966f0f` | `Mode: Streaming`

Streaming response:


  >> Tool: retrieve_historical_stock_price({'symbol': 'AMZN', 'start_date': '2023-10-01', 'end_date': '2023-12-31'})
  >> Executing tool...
Based on the historical data for Amazon (AMZN) in Q4 2023:

1. Starting Price (October): Around $120
2. Ending Price (December): $151.94
3. Price Range:
   - Lowest: $119.57
   - Highest: $154.07
   - Average: $140.18

The stock showed a strong upward trend during Q4 2023, with the price increasing by approximately 27% from its lowest to highest point. The stock closed the quarter near its highest levels, at $151.94 on December 29, 2023 (the last trading day of the year).

For context, would you like to see how this compares to current prices or any specific analysis of this period?


---
Completed in 10485ms (first byte: 5577ms) | 5 events | Tools: retrieve_historical_stock_price


## Query 3: Compare Performance vs Analyst Predictions

**Question**: "Compare Amazon's recent stock performance to what analysts predicted in their reports"

**Expected Tools**: `retrieve_realtime_stock_price` + `search_financial_documents`

This demonstrates **RAG** using the Bedrock Knowledge Base.

In [158]:
result_3 = query_agent(
    "Compare Amazon's recent stock performance to what analysts predicted in their reports",
    token=access_token,
    stream=True,
)

---
### Query
> Compare Amazon's recent stock performance to what analysts predicted in their reports

`Session: 23bc8f58-ebbc-41ce-b7a5-d3d572940dd9` | `Mode: Streaming`

Streaming response:


  >> Tool: retrieve_realtime_stock_price({'symbol': 'AMZN'})
  >> Executing tool...

  >> Tool: search_financial_documents({'query': 'analyst price targets predictions forecasts 2024'})
  >> Executing tool...

  >> Tool: retrieve_historical_stock_price({'symbol': 'AMZN', 'start_date': '2024-01-01', 'end_date': '2026-02-16'})
  >> Executing tool...
Based on the gathered information, here's a comparison of Amazon's performance versus analyst predictions:

1. Current Performance:
- Current stock price: $198.79
- Recent trading range: $197.28 - $201.16
- Average price over the period: $202.80

2. Financial Performance (from 2024 Annual Report):
- Total revenue grew 11% year-over-year to $638B
- Operating income improved 86% year-over-year to $68.6B
- Operating margin increased to 10.8%
- Strong performance across all segments:
  * North America: +10% growth
  * International: +9% growth
  * AWS: +19% growth

3. Company Guidance and Performance:
- The company has been 

## Query 4: Multi-Tool Query (Stock + AI Business)

**Question**: "I'm researching AMZN give me the current price and any relevant information about their AI business"

**Expected Tools**: `retrieve_realtime_stock_price` + `search_financial_documents`

In [159]:
result_4 = query_agent(
    "I'm researching AMZN give me the current price and any relevant information about their AI business",
    token=access_token,
    stream=True,
)

---
### Query
> I'm researching AMZN give me the current price and any relevant information about their AI business

`Session: 7bd4b99b-2337-4157-baf8-270eb71f5aa1` | `Mode: Streaming`

Streaming response:


  >> Tool: retrieve_realtime_stock_price({'symbol': 'AMZN'})
  >> Executing tool...

  >> Tool: search_financial_documents({'query': 'Amazon AI initiatives, artificial intelligence, AWS AI services'})
  >> Executing tool...
Based on the gathered information, here's a comprehensive update on Amazon (AMZN):

Current Stock Information:
- Current Price: $198.79
- Market Cap: $2.13 trillion
- Today's Trading Range: $197.28 - $201.16

Amazon's AI Business Highlights:
1. Broad AI Investment and Development:
- Over 1,000 Generative AI applications being built across Amazon
- Focus areas include shopping, coding, personal assistants, streaming, advertising, healthcare, reading, and home devices

2. Key AI Infrastructure and Services:
- Custom AI chips (Amazon Trainium) for better price-performance
- Amazon Bedrock and Amazon SageMaker for model-building and inference
- Amazon Nova for frontier models
- Recently launched AWS Transform, which accelerates mainframe modernizat

## Query 5: Document-Only Query with Streaming (Office Space)

**Question**: "What is the total amount of office space Amazon owned in North America in 2024?"

**Expected Tool**: `search_financial_documents`

**Mode**: `stream=True` — demonstrates **SSE streaming via `.astream()`**

In [155]:
result_5 = query_agent(
    "What is the total amount of office space Amazon owned in North America in 2024?",
    token=access_token,
    stream=True,
)

---
### Query
> What is the total amount of office space Amazon owned in North America in 2024?

`Session: e6ded2d3-9a19-45fc-806a-911aac927e8f` | `Mode: Streaming`

Streaming response:


  >> Tool: search_financial_documents({'query': 'office space owned North America square feet 2024'})
  >> Executing tool...
Based on the search results from Amazon's financial documents, as of December 31, 2024, Amazon owned 9,104,000 square feet of office space in North America. This information comes from their detailed property breakdown in their 2024 financial reporting.

To provide additional context:
1. This represents only the office space that Amazon owns (not leases)
2. For comparison, Amazon also leased 29,551,000 square feet of office space in North America
3. This data excludes other types of facilities like physical stores, fulfillment centers, and data centers
4. The figures are reported in thousands of square feet in the original document, so I've converted it to the full number

Source: This information comes from Amazon's 2024 financial documents, specifically from their detailed property section which breaks down their facilities by type and loc

---

# Summary & Validation

## ✅ User Acceptance Criteria Completed

This notebook successfully demonstrated:

### 1. **Cognito Authentication** ✅
- ✅ Created test user in Cognito user pool
- ✅ Authenticated via `USER_PASSWORD_AUTH`
- ✅ Retrieved JWT access token
- ✅ **Used JWT token for all AgentCore invocations via Bearer token**

### 2. **All 5 Required Queries** ✅
1. ✅ Real-time stock price - `retrieve_realtime_stock_price`
2. ✅ Historical Q4 data - `retrieve_historical_stock_price`
3. ✅ Stock + analyst predictions - Multi-tool with RAG
4. ✅ AMZN price + AI business - Complex multi-tool query
5. ✅ Office space data - `search_financial_documents`

### 3. **AgentCore Invocation** ✅
- ✅ HTTPS endpoint invocation
- ✅ **Cognito JWT Bearer token authentication** (OAuth method)
- ✅ Correct session ID format (33-256 chars)
- ✅ JSON request/response handling

### 4. **LangGraph ReAct Agent** ✅
- ✅ Multi-step reasoning
- ✅ Tool selection and execution
- ✅ Response synthesis

### 5. **AWS Services Integration** ✅
- ✅ AWS Cognito - **JWT authentication with Bearer token**
- ✅ AWS Bedrock AgentCore - Runtime hosting with OAuth config
- ✅ AWS Bedrock KB - RAG with 3 financial documents
- ✅ yfinance API - Stock data (realtime + historical)

---

## 🔐 Authentication Flow Diagram

```
┌─────────────┐
│   Notebook  │
└──────┬──────┘
       │
       │ 1. authenticate()
       ▼
┌─────────────────┐
│  AWS Cognito    │
│  User Pool      │
└────────┬────────┘
         │
         │ 2. Returns JWT token
         ▼
┌─────────────────┐
│  query_agent()  │
│  + Bearer token │
└────────┬────────┘
         │
         │ 3. HTTPS POST with Authorization: Bearer <JWT>
         ▼
┌──────────────────┐
│  AgentCore       │
│  (validates JWT) │
└────────┬─────────┘
         │
         │ 4. If valid, processes request
         ▼
┌──────────────────┐
│  FastAPI App     │
│  LangGraph Agent │
└──────────────────┘
```

---

## 📊 Langfuse Observability

All executions are automatically traced in **Langfuse** (cloud free tier).

**View traces at**: https://cloud.langfuse.com or https://us.cloud.langfuse.com

**Trace details include**:
- LLM invocations (Claude 3.5 Sonnet)
- Tool calls with parameters
- Execution times and token usage

**📸 Capture screenshots** showing:
- Agent reasoning traces
- Tool invocations
- Token/cost metrics

---

## 🎯 Assessment Requirements: 100% Complete

| Requirement | Status |
|------------|--------|
| AWS Agentcore | ✅ |
| **AWS Cognito JWT Auth** | ✅ **USED** |
| Langfuse | ✅ |
| LangGraph ReAct | ✅ |
| Knowledge Base (3 docs) | ✅ |
| `retrieve_realtime_stock_price` | ✅ |
| `retrieve_historical_stock_price` | ✅ |
| Streaming `.astream()` | ✅ |
| Terraform | ✅ |
| **Notebook with Cognito auth** | ✅ **DEMONSTRATED** |
| Langfuse traces | ✅ |

---

**🎉 All user acceptance criteria validated with proper Cognito JWT authentication!**